# Calculate Hours of Daylight

**Purpose of script:**

Calculating the hours of sunlight for a given latitude and day of the year
- In: Interpolated mw file to have the coordinate grid (tif)
- Out: DataFrame with calculated hours of sunlight per latitude and date (parquet)

*Note:*
- 0 means no hours of sunlight == total darkness all day long, 24 means only sunlight == total brightness all day long
- Formula taken from the following paper: Forsythe, William C., et al. "A model comparison for daylength as a function of latitude and day of year." Ecological Modelling 80.1 (1995): 87-95. --> https://www.researchgate.net/profile/Randal-Stahl/publication/216811293_A_Model_Comparison_for_Daylength_as_a_Function_of_Latitude_and_Day_of_Year/links/5b153fbca6fdcc4611e2b08f/A-Model-Comparison-for-Daylength-as-a-Function-of-Latitude-and-Day-of-Year.pdf

In [1]:
import pyproj
import pandas as pd
from math import asin, cos, atan, tan, pi, acos, sin 

import xarray

In [2]:
mw_path = r"../Data/microwave-rs/mw_interpolated/2017-05-01_mw.tif"
out_path = r"../Data/daylight_data/"

In [3]:
# Define the source and destination coordinate reference systems
src_crs = pyproj.CRS.from_epsg(3413)  # WGS84 (longitude, latitude)
dst_crs = pyproj.CRS.from_epsg(4326)  # Web Mercator (used by most online maps)

# Define the transformer object
transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)

df = xarray.open_dataarray(mw_path).to_dataframe().reset_index()[["y", "x"]]

# Convert all coordinates at once
lats, longs = transformer.transform(df["x"], df["y"])

# only need latitude values (y) for daylight calculation
df["lat"] = lats

In [4]:
df

,y,x,lat
0,-662500.0,-636500.0,81.533891
1,-662500.0,-635500.0,81.540251
2,-662500.0,-634500.0,81.546605
3,-662500.0,-633500.0,81.552955
4,-662500.0,-632500.0,81.559299
...,...,...,...
3893301,-3324500.0,820500.0,59.128763
3893302,-3324500.0,821500.0,59.126701
3893303,-3324500.0,822500.0,59.124636
3893304,-3324500.0,823500.0,59.122570


In [21]:
df_final = df[["x", "y"]]

# create a list of dates for random year since we assume the solar duration to have no significant change over the years
datelist = pd.date_range("2017-05-01", "2017-10-31")

previous_month = 5
for date in datelist:

    print(date)

    # store to parquet file if month changes
    if date.month != previous_month:
        df_final.to_parquet(out_path + f"hours_of_daylight_month{previous_month}.parquet")
        del df_final
        df_final = df[["x", "y"]]
        previous_month = date.month
    

    # function to calculate the hours of daylight for a given latitude and day of the year
    def calculate_hours_of_daylight(L, J):

        # 1. Predicting the revolution angle (R) from the day of the year (J). 
        R = 0.2163108 + 2 * atan(0.9671396 * tan(.00860 * (J - 186)))
        # 2. Predicting the sun's declination angle.
        P = asin(0.39795 *cos(R)) 
        # 3. Calculate parameter to differentiate between complete darkness (<= -1) and complete light (>= 1)
        param = sin((0.8333 * pi / 180) + sin(L * pi / 180) * sin(P)) / (cos(L * pi / 180) * cos(P))
        # 4. Predicting daylength (D) (plus twilight) from latitude (L) and the sun's declination angle (P).
        if param <= -1:
            D = 0
        elif param >= 1:
            D = 24
        else:
            D = 24 - (24 / pi) * acos(param)
        
        return D

    
    df_temp = df.copy()

    # get the day of the year
    J = date.dayofyear

    # calculate the hours of daylight for each latitude
    df_temp[f"daylight_day_{J}"] = df_temp["lat"].apply(lambda L: calculate_hours_of_daylight(L, J))
    # drop lat column
    df_temp = df_temp.drop(columns="lat")

    # append to final dataframe
    df_final = pd.concat([df_final, df_temp[[f"daylight_day_{J}"]]], axis=1)

# save as parquet
df_final.to_parquet(out_path + f"hours_of_daylight_month{previous_month}.parquet")

2017-05-01 00:00:00
2017-05-02 00:00:00
2017-05-03 00:00:00
2017-05-04 00:00:00
2017-05-05 00:00:00
2017-05-06 00:00:00
2017-05-07 00:00:00
2017-05-08 00:00:00
2017-05-09 00:00:00
2017-05-10 00:00:00
2017-05-11 00:00:00
2017-05-12 00:00:00
2017-05-13 00:00:00
2017-05-14 00:00:00
2017-05-15 00:00:00
2017-05-16 00:00:00
2017-05-17 00:00:00
2017-05-18 00:00:00
2017-05-19 00:00:00
2017-05-20 00:00:00
2017-05-21 00:00:00
2017-05-22 00:00:00
2017-05-23 00:00:00
2017-05-24 00:00:00
2017-05-25 00:00:00
2017-05-26 00:00:00
2017-05-27 00:00:00
2017-05-28 00:00:00
2017-05-29 00:00:00
2017-05-30 00:00:00
2017-05-31 00:00:00
